### Original dictionary
The Wordle dictionary, sorted alphabetically, is stored in wordle_dictionary.py

In [1]:
from wordle_vocab import dictionary

import pandas as pd
import numpy as np
import json

## Create Popularity Scores

You need to first run `scrape_word_popularities.py` to obtain a csv file containing all of the popularity scores for every word in Wordle's vocabulary from Google Books. We then rework that into a nice dictionary of scores that can be used by Wordle bot.

Current plan is to take the average popularity score of each word (case insensitive) from the past ten years. That will be a small offset we add to the entropy score of each word in entropy_guesser.

In [2]:
df=pd.read_csv("word_popularity_data.csv.zip",compression="zip")

In [3]:
def score_averager(x,n_years=10):
    return np.mean(x[-n_years:])

In [4]:
df["average_popularity"]=df.apply(score_averager,axis=1)

In [5]:
idx=df["case"]=="Insensitive"
columns=["word","case","average_popularity"]
df.loc[idx,columns]

,word,case,average_popularity
0,aahed,Insensitive,4.629283e-08
1,aalii,Insensitive,4.632601e-10
3,aargh,Insensitive,4.016478e-08
5,aarti,Insensitive,8.236386e-08
7,abaca,Insensitive,4.356348e-08
...,...,...,...
25738,zuzim,Insensitive,7.567169e-09
25740,zygal,Insensitive,9.703162e-10
25742,zygon,Insensitive,4.902521e-08
25744,zymes,Insensitive,8.497195e-09


In [6]:
df.loc[idx,"word"].str.lower()

0        aahed
1        aalii
3        aargh
5        aarti
7        abaca
         ...  
25738    zuzim
25740    zygal
25742    zygon
25744    zymes
25746    zymic
Name: word, Length: 13038, dtype: object

In [7]:
pop_dict=dict(zip(
    df.loc[idx,"word"].str.lower().values,
    df.loc[idx,"average_popularity"].values
)
)

In [8]:
# Serialize data into file:
json.dump( pop_dict, open( "popularity_dict.json", 'w' ) )

The contents of the above JSON file can just be copy and pasted into WordleBot/wordle_dictionary.py as `popularity_dict`

## Scored dictionary

First let's create a list of words from this data, sorted by word popularity

In [9]:
pop_list = [x[0] for x in sorted(pop_dict.items(), key=lambda x: -x[1])] # List of dictionary words sorted from most to least popular

Here are the 100 most popular words from the dictionary: 

In [10]:
print(pop_list[:100])

['which', 'their', 'there', 'would', 'about', 'other', 'could', 'these', 'first', 'after', 'where', 'those', 'being', 'right', 'while', 'world', 'years', 'still', 'under', 'state', 'three', 'think', 'never', 'again', 'might', 'going', 'place', 'found', 'great', 'every', 'since', 'water', 'small', 'based', 'human', 'power', 'asked', 'often', 'order', 'women', 'house', 'using', 'point', 'given', 'until', 'group', 'table', 'large', 'later', 'study', 'press', 'level', 'early', 'shall', 'words', 'among', 'night', 'times', 'value', 'young', 'light', 'model', 'thing', 'white', 'woman', 'least', 'means', 'along', 'whole', 'black', 'heart', 'known', 'child', 'hands', 'above', 'sense', 'local', 'taken', 'death', 'voice', 'began', 'court', 'money', 'party', 'doing', 'front', 'close', 'terms', 'heard', 'short', 'class', 'space', 'field', 'third', 'blood', 'clear', 'quite', 'story', 'cases', 'major']


And the 100 least popular: 

In [11]:
print(pop_list[-100:])

['cowks', 'bwazi', 'weamb', 'jarps', 'atocs', 'cauks', 'byded', 'skaws', 'twals', 'yechs', 'kophs', 'cawks', 'chynd', 'sowff', 'inorb', 'syboe', 'buats', 'spyal', 'boygs', 'golps', 'tehrs', 'tozed', 'eilds', 'upjet', 'ikans', 'yitie', 'uraos', 'noups', 'souct', 'bedye', 'zhomo', 'gyeld', 'gawcy', 'ylkes', 'birsy', 'nemns', 'hawms', 'mozed', 'wudus', 'wauff', 'tozes', 'gymps', 'ylems', 'apoop', 'sdein', 'peeoy', 'zedas', 'pioys', 'fusks', 'hoaed', 'ryked', 'pyxed', 'zimbs', 'peghs', 'dsomo', 'atoks', 'advew', 'hohed', 'skyrs', 'wembs', 'avyze', 'awdls', 'azygy', 'byked', 'byrls', 'dsobo', 'dzhos', 'eevns', 'egmas', 'ennog', 'euked', 'evhoe', 'ewked', 'gowfs', 'hiois', 'humfs', 'hwyls', 'koaps', 'kuzus', 'nabks', 'odyls', 'omovs', 'phpht', 'poupt', 'pyins', 'qapik', 'qophs', 'sdayn', 'skyfs', 'snebs', 'sowfs', 'syped', 'tiyns', 'viffs', 'voema', 'vutty', 'xysts', 'yesks', 'yrivd', 'zexes']


Come on, there's no way some of these are words! Still, Wordle allows them so we'll include them, I guess...

### Create letter scores
The score for each letter is the total number of times it occurs in the dictionary

In [12]:
pop_list_reverse = pop_list[::-1] # so that final dictionary will be sorted alphabetically within equal scores

str_dictionary = ''.join(pop_list_reverse)
alphabet = 'abcdefghijklmnopqrstuvwxyz'
alphabet_list = [a for a in alphabet]
letter_scores_list = [str_dictionary.count(letter) for letter in alphabet]

# Score of each letter is just the number of times it appears in the dictionary
letter_scores = dict(zip(alphabet_list,letter_scores_list))

### Here are the letter scores this generates

In [13]:
letter_scores = {'a': 5990, 'b': 1627, 'c': 2028, 'd': 2453, 'e': 6662, 'f': 1115, 'g': 1644, 'h': 1760, 'i': 3759, 'j': 291, 'k': 1505, 'l': 3371, 'm': 1976, 'n': 2952, 'o': 4438, 'p': 2019, 'q': 112, 'r': 4158, 's': 6665, 't': 3295, 'u': 2511, 'v': 694, 'w': 1039, 'x': 288, 'y': 2074, 'z': 434}

### Create dictionary scores
For each word in the dictionary, the score is the sum of its *unique* letter scores

In [14]:
# Score of a word is the sum of the *unique* letter scores in each word
dictionary_scores_list = [sum([letter_scores[letter] for letter in set(word)]) for word in pop_list_reverse]
dictionary_scores = dict(zip(pop_list_reverse,dictionary_scores_list))
scored_dictionary = sorted(dictionary_scores, key=dictionary_scores.get)
scored_dictionary.reverse() # Sort by score *descending*

Here are the 100 highest-scoring words:

In [15]:
print(scored_dictionary[:100])

['arose', 'aeros', 'soare', 'raise', 'arise', 'serai', 'aesir', 'reais', 'aloes', 'stoae', 'toeas', 'laser', 'earls', 'arles', 'reals', 'rales', 'lares', 'lears', 'seral', 'laers', 'rates', 'tears', 'stare', 'aster', 'taser', 'tares', 'teras', 'stear', 'resat', 'reast', 'arets', 'strae', 'earst', 'aeons', 'aisle', 'snare', 'earns', 'nears', 'nares', 'saner', 'reans', 'anise', 'saine', 'isnae', 'ureas', 'aures', 'ursae', 'urase', 'least', 'tales', 'steal', 'slate', 'stale', 'tesla', 'stela', 'taels', 'setal', 'teals', 'salet', 'leats', 'reads', 'dares', 'dears', 'arsed', 'rased', 'sared', 'eards', 'paseo', 'psoae', 'osier', 'lanes', 'leans', 'slane', 'neals', 'elans', 'antes', 'stane', 'nates', 'neats', 'stean', 'etnas', 'years', 'ayres', 'sayer', 'arsey', 'eyras', 'resay', 'ideas', 'aside', 'aides', 'acres', 'races', 'cares', 'scare', 'carse', 'serac', 'acers', 'escar', 'scrae', 'spare']


The entire `scored_dictionary` is included in ```WordleBot/wordle_dictionary.py```